# Data Cleaning for Food Environment Project


#### Imports and Defining Directories

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define directory where data is located
data_dir =  "/content/drive/My Drive/DS 5100 Food Environment Project/FoodEnvironmentAtlas/"

# Display files located there
!ls  "/content/drive/My Drive/DS 5100 Food Environment Project/FoodEnvironmentAtlas"

ReadMeFile2020.txt	   SupplementalDataCounty.csv  VariableList.gsheet
StateAndCountyData.csv	   SupplementalDataState.csv
StateAndCountyData.gsheet  VariableList.csv


#### Perform Data Cleaning

In [ ]:
# Read in state and county data
df = pd.read_csv(data_dir + 'StateAndCountyData.csv')
df.head()

,FIPS,State,County,Variable_Code,Value
0,1001,AL,Autauga,LACCESS_POP10,18428.439690
1,1001,AL,Autauga,LACCESS_POP15,17496.693040
2,1001,AL,Autauga,PCH_LACCESS_POP_10_15,-5.056026
3,1001,AL,Autauga,PCT_LACCESS_POP10,33.769657
4,1001,AL,Autauga,PCT_LACCESS_POP15,32.062255


In [ ]:
# Create instance of DataCleaning class
clean = DataCleaning(df)

In [ ]:
# Perform all data cleaning steps
clean.full_data_cleaning()

In [ ]:
# Save cleaned data back to google drive
# Define directory where to save cleaned data
data_dest =  "/content/drive/My Drive/DS 5100 Food Environment Project/Data_Cleaned/"

# Save state and county dataframes
clean.df_state.to_csv(data_dest + 'State_Level_Data.csv', index=False)
clean.df_county.to_csv(data_dest + 'County_Level_Data.csv', index=False)

#### Class Definition 

Create data cleaning class to perform all necessary data cleaning tasks.

In [ ]:
# Create class to perform data cleaning
class DataCleaning:
  # Decription of class and fields

  def __init__(self, df):
    self.df = df.copy() # Original dataframe
    # copy so that changes within class doesn't change original

  def full_data_cleaning(self):
    '''
    Method to perform all necessary cleaning tasks. Each distinct task/step
    is defined as its own method and called sequentially.
    '''
    self.clean_state_column()
    self.prep_fips_lookup_table()
    self.clean_county_column()
    self.reformat_data()
    self.split_state_county_data()

  def clean_state_column(self):
    # Remove extra whitespace on some state names
    self.df['State'] = self.df['State'].str.strip()

  def prep_fips_lookup_table(self):
    # Webscrape fips table
    self.webscrape_fips_lookup()
    # Add missing fips codes to the table
    self.add_missing_fips()

  def webscrape_fips_lookup(self):
    '''
    Function to webscrape fips lookup table
    '''
    # Define header
    headers = {'user-agent': 'UVA Project (pkx2ec@virginia.edu) (Language=Python 3.8.2; Platform=Macintosh; Intel Mac OS X 11_5_2)'} 
    # Specify URL
    URL = 'https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697'
    # Access HTML content 
    r = requests.get(URL, headers= headers)
    # Parse HTML content
    soup = BeautifulSoup(r.content, 'html5lib')
    # Find data of interest
    table = soup.find('div', attrs = {'class':'centerColImg'})  
    # Create empty dataframe to save FIPS info
    self.fips_table = pd.DataFrame(columns = ['FIPS', 'County', 'State'])
    # Add each FIPS code to dataframe
    first_row = True
    for row in table.findAll('tr'):
      # First row doesn't have the info we want so skip it
      if first_row:
        first_row = False
      else:
        row_entries = row.findAll('td')
        row_text = [i.text for i in row_entries]
        self.fips_table.loc[len(self.fips_table)] = row_text
    # Change type of fips table to int so we join it with our data
    self.fips_table['FIPS'] = self.fips_table['FIPS'].astype(int)

  def add_missing_fips(self):
    # Update fips lookup table to add any fips in df, but not fips_table
    # List of fips codes in df and fips_table
    original_fips = self.df.FIPS.unique()
    new_fips = self.fips_table.FIPS.unique()
    # For any missing fips, add the info from the first record in df
    for i in original_fips:
      if i not in new_fips:
        self.fips_table.loc[len(self.fips_table.index)] = self.df.loc[self.df['FIPS']==i,['FIPS', 'County', 'State']].iloc[0]

  def clean_county_column(self):
    # Webscrape fips lookup table
    self.prep_fips_lookup_table()
    # Add any missing fips codes to the lookup table
    self.add_missing_fips()
    # Drop original state and county columns (otherwise we'll have 2 columns with the same name)
    self.df.drop(['State', 'County'], axis=1, inplace = True)
    # Join together fips lookup table
    # Left join so we don't lose any data from our original table
    self.df = self.df.merge(self.fips_table, on = ['FIPS'], how = 'left')

  def reformat_data(self):
    # Re-format so variables are across the columns not adding rows in the "Variable Code" column
    self.pivot = pd.pivot_table(self.df, index = ['FIPS', 'State', 'County'], columns = 'Variable_Code', values = 'Value')
    self.pivot.reset_index(inplace=True)

  def split_state_county_data(self):
    '''
    Some FIPS codes (1-56) are state level, all others are at the county level. 
    In addition, some of the variables in the dataset are at the state level
    (corresponding to these 1-56 FIPS codes) and thus will be missing for
    all the remaining county level FIPS codes. Other variables are at a county
    level so those will be missing for the state level FIPS codes.

    This method splits the data into a state level and county level dataframe
    and removes variables with all missing values in the resulting dataframes.
    '''
    # Split data based on FIPS code
    self.df_state = self.pivot[self.pivot.FIPS <= 56] 
    self.df_county = self.pivot[self.pivot.FIPS > 56]
    # Drop columns with all missing values
    self.df_state = self.df_state.dropna(axis=1, how='all')
    self.df_county = self.df_county.dropna(axis=1, how='all')



#### Testing Class

In [ ]:
import unittest

In [ ]:
# Create class that inherits from unittest.TestCase
class DataCleaning_Test(unittest.TestCase):
  def setUp(self):
    # Set up
    # Create class with small dataset to use for testing
    sample_df = pd.DataFrame({'FIPS':[1,1,20,20,1001,1001,99999], 
                              'State': ['AL','AL','KS','KS ', 'AL', 'AL ', 'ZZ'],
                              'County': ['Total','Total','Total','Total','Autauga','Autauga County','Fake'],
                              'Variable_Code':['Var1', 'Var2', 'Var1', 'Var2', 'Var1', 'Var2', 'Var1'],
                              'Value': [5,np.nan,10,np.nan,np.nan,20,np.nan]})
    self.clean1 = DataCleaning(sample_df)

  def test_clean_state_col(self):
    # Is any white space removed from states?
    # Clean state column
    self.clean1.clean_state_column()
    # Test: using assetEqual() method
    self.assertEqual(list(self.clean1.df.State), ['AL','AL','KS','KS', 'AL', 'AL', 'ZZ']) 

  


In [ ]:
# if __name__ == '__main__':
#     unittest.main() 
unittest.main(argv=[''],exit=False) 

.
----------------------------------------------------------------------
Ran 1 test in 0.006s

OK
